In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandexo.engine.justdoit as jdi # THIS IS THE HOLY GRAIL OF PANDEXO
import numpy as np
import os

## Editting Inputs

### Step 1) Load in a blank exoplanet dictionary

To start, load in a blank exoplanet dictionary with empty keys. You will fill these out for yourself in the next step. 

In [2]:
exo_dict = jdi.load_exo_dict()

#### Edit exoplanet observation inputs
Editting each keys are annoying. But, do this carefully or it could result in nonsense runs

In [3]:
exo_dict['observation']['sat_level'] = 80    #saturation level in percent of full well 
exo_dict['observation']['noccultations'] = 2 #number of transits 
exo_dict['observation']['R'] = None          #fixed binning. I usually suggest ZERO binning.. you can always bin later 
                                             #without having to redo the calcualtion
exo_dict['observation']['fraction'] = 1.0    #fraction of time in transit versus out = in/out
exo_dict['observation']['noise_floor'] = 0   #this can be a fixed level or it can be a filepath 
                                             #to a wavelength dependent noise floor solution (units are ppm)

#### Edit exoplanet star inputs
Note... If you select 'phoenix' you **do not** have to provide a starpath, w_unit or f_unit, but you **do** have to provide a temp, metal and logg. If you select 'user' you **do not** need to provide a temp, metal and logg, but you **do** need to provide units and starpath. 

In [4]:
exo_dict['star']['type'] = 'phoenix'        #phoenix or user (if you have your own)
exo_dict['star']['mag'] = 8.0               #magnitude of the system
exo_dict['star']['ref_wave'] = 1.25         #For J mag = 1.25, H = 1.6, K =2.22.. etc (all in micron)
exo_dict['star']['temp'] = 5500             #in K 
exo_dict['star']['metal'] = 0.0             # as log Fe/H
exo_dict['star']['logg'] = 4.0              #log surface gravity cgs

#### Edit exoplanet planet inputs

In [5]:
exo_dict['planet']['exopath'] = '/Users/nbatalh1/Desktop/Simulations/wasp12b.txt'
exo_dict['planet']['w_unit'] = 'cm'                      #other options include "um", "Angs", "secs" (for phase curves)
exo_dict['planet']['f_unit'] = 'rp^2/r*^2'               #other options are 'fp/f*' 
exo_dict['planet']['transit_duration'] = 2.0*60.0*60.0   #transit duration in seconds

### Step 2) Load in instrument dictionary (OPTIONAL)
 
Step 2 is optional because PandExo has the functionality to automatically load in instrument dictionaries. Skip this if you plan on observing with one of the following and want to use the subarray with the smallest frame time and the readout mode with 1 frame/1 group (standard): 
- NIRCam F444W
- NIRSpec Prism
- NIRSpec G395M
- NIRSpec G395H
- NIRSpec G235H
- NIRSpec G235M
- NIRCam F322W
- NIRSpec G140M
- NIRSpec G140H
- MIRI LRS
- NIRISS SOSS

In [6]:
jdi.print_instruments()

Choose from the following:
['NIRCam F444W', 'NIRSpec Prism', 'NIRSpec G395M', 'NIRCam F322W2', 'NIRSpec G395H', 'NIRSpec G235H', 'NIRSpec G235M', 'NIRSpec G140M', 'NIRSpec G140H', 'MIRI LRS', 'WFC3 G141', 'NIRISS SOSS']


In [7]:
inst_dict = jdi.load_mode_dict('MIRI LRS')

#loading in instrument dictionaries allow you to personalize some of  
#the fields that are predefined in the templates. The templates have 
#the subbarays with the lowest frame times and the readmodes with 1 frame per group. 
#if that is not what you want. change these fields

inst_dict["configuration"]["detector"]["ngroup"] = 'optimize'   #running "optimize" will select the maximum 
                                                                #possible groups before saturation. 
                                                                #You can also write in any integer between 2-65536

inst_dict["configuration"]["detector"]["subarray"] = 'sub512'   #change the subbaray
inst_dict["configuration"]["detector"]["readmode"] = 'nrs'      #change the readout pattern


## Running PandExo

You have **four options** for running PandExo. All of them are accessed through attribute **jdi.run_pandexo**. See examples below. 

`    jdi.run_pandexo(exo, inst, param_space = 0, param_range = 0,save_file = True,
                            output_path=os.getcwd(), output_file = '')`



### Option 1- Run single instrument mode, single planet
If you forget which instruments are available run **jdi.print_isntruments()** and pick one 

In [8]:
jdi.print_instruments()

Choose from the following:
['NIRCam F444W', 'NIRSpec Prism', 'NIRSpec G395M', 'NIRCam F322W2', 'NIRSpec G395H', 'NIRSpec G235H', 'NIRSpec G235M', 'NIRSpec G140M', 'NIRSpec G140H', 'MIRI LRS', 'WFC3 G141', 'NIRISS SOSS']


In [ ]:
result = jdi.run_pandexo(exo_dict,['NIRCam F322W2'])

### Option 2- Run single instrument mode (with user dict), single planet
This is the same thing as option 1 but instead of feeding it a list of keys, you can feed it a instrument dictionary (this is for users who wanted to simulate something NOT pre defined within pandexo)

In [ ]:
inst_dict = jdi.load_mode_dict('NIRSpec G140H')
#personalize subarray
inst_dict["configuration"]["detector"]["subarray"] = 'sub2048'
result = jdi.run_pandexo(exo_dict, inst_dict)

### Option 3- Run several modes, single planet
Use several modes from **print_isntruments()** options. 

In [ ]:
#choose select 
result = jdi.run_pandexo(exo_dict,['NIRSpec G140M','NIRSpec G235M','NIRSpec G395M'],
               output_file='three_nirspec_modes.p')
#run all 
#result = jdi.run_pandexo(exo_dict, ['RUN ALL'], save_file = False)

### Option 4- Run single mode, several planet cases
Use a single modes from **print_isntruments()** options. But explore parameter space with respect to **any** parameter in the exo dict. The example below shows how to loop over several planet models

You can loop through anything in the exoplanet dictionary. It will be planet, star or observation followed by whatever you want to loop through in that set. 

i.e. planet+exopath, star+temp, star+metal, star+logg, observation+sat_level.. etc

In [9]:
#looping over different exoplanet models 
jdi.run_pandexo(exo_dict, ['NIRCam F444W'], param_space = 'planet+exopath',
                param_range = os.listdir('/Users/nbatalha1/Desktop/all_my_models_here'),
               output_path = '/Users/nbatalh1/Desktop/JWSTFUN')

#looping over different stellar temperatures 
jdi.run_pandexo(exo_dict, ['NIRCam F444W'], param_space = 'star+temp',
                param_range = np.linspace(5000,8000,2),
               output_path = '/Users/nbatalh1/Desktop/JWSTFUN')

#looping over different saturation levels
jdi.run_pandexo(exo_dict, ['NIRCam F444W'], param_space = 'observation+sat_level',
                param_range = np.linspace(.5,1,5),
               output_path = '/Users/nbatalh1/Desktop/JWSTFUN')


Running through exo parameters in parallel: star+temp
Optimization Reqested: Computing Duty Cycle
Optimization Reqested: Computing Duty Cycle
Finished Duty Cycle Calc
Starting Out of Transit Simulation
Finished Duty Cycle Calc
Starting Out of Transit Simulation
End out of Transit
Starting In Transit Simulation
End In Transit
End out of Transit
Starting In Transit Simulation
End In Transit
Running through exo parameters in parallel: observation+sat_level
Optimization Reqested: Computing Duty Cycle
Optimization Reqested: Computing Duty Cycle
Optimization Reqested: Computing Duty Cycle
Optimization Reqested: Computing Duty Cycle
Finished Duty Cycle Calc
Starting Out of Transit Simulation
Finished Duty Cycle Calc
Starting Out of Transit Simulation
Finished Duty Cycle Calc
Starting Out of Transit Simulation
Finished Duty Cycle Calc
Starting Out of Transit Simulation
End out of Transit
Starting In Transit Simulation
End In Transit
End out of Transit
Starting In Transit Simulation
End In Tran

[{'0.5': {'FinalSpectrum': {'error_w_floor': array([  2.72142719e+00,   2.08678381e+00,   1.94123191e+00, ...,
             2.16410971e-03,   2.32731755e-03,   3.02143870e-03]),
    'spectrum': array([ 0.0136671 ,  0.01366708,  0.01366706, ...,  0.01399279,
            0.01400599,  0.01401919]),
    'spectrum_w_rand': array([-1.10669861,  2.42478615, -1.06518935, ...,  0.01865386,
            0.01529371,  0.01781109]),
    'wave': array([ 3.7  ,  3.701,  3.702, ...,  4.996,  4.997,  4.998])},
   'OriginalInput': {'model_spec': array([ 0.01370944,  0.0137092 ,  0.01370907,  0.01370905,  0.01370915,
            0.01370937,  0.01370972,  0.01371022,  0.01371086,  0.01371166,
            0.01371263,  0.01371377,  0.0137151 ,  0.01371662,  0.01371834,
            0.01372029,  0.01372246,  0.01372487,  0.01372754,  0.01373048,
            0.01373371,  0.01373724,  0.01374108,  0.01374527,  0.01374983,
            0.01375481,  0.01376023,  0.01376616,  0.01377265,  0.01377979,
            0.0